# Exploratory Data Analysis of Vitamin D-Related Transcriptional Signatures

This notebook presents the exploratory data analysis (EDA) of transcriptional responses to vitamin D and its analogs, based on data from the LINCS L1000 dataset. The dataset includes gene expression signatures from human cell lines exposed to different vitamin D-related compounds across various doses and time points.

The goal of this analysis is to characterize the diversity and distribution of the selected signatures, examine the experimental conditions (cell lines, doses, exposure times), and identify preliminary patterns in gene expression profiles that may inform subsequent modeling and biological interpretation.

In [ ]:
# Import libraries for data manipulation, visualization, and dimensionality reduction
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [ ]:
# Visualization settings
sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (8, 6)

In [ ]:
# Paths to pre-filtered files
EXPR_PATH = "../processed_data/vitD_expression_matrix.csv"
META_PATH = "../processed_data/siginfo_vitD_filtered.csv"

# Load expression matrix (genes x signatures)
exp_df = pd.read_csv(EXPR_PATH, index_col=0)

# Load metadata associated with the signatures
meta_df = pd.read_csv(META_PATH, index_col="sig_id")

# Display shapes
print("Expression matrix shape:", exp_df.shape)
print("Metadata shape:", meta_df.shape)


In [ ]:
# Confirm all signature IDs in metadata are present in the expression matrix
assert set(meta_df.index).issubset(set(exp_df.columns)), "Some sig_id in metadata are missing from expression matrix"

# Reorder expression matrix columns to match metadata index
exp_df = exp_df[meta_df.index]

# Final confirmation
print("Column order aligned:", all(exp_df.columns == meta_df.index))

In [ ]:
# Display a subset of the expression matrix to inspect gene-signature structure
display(exp_df.iloc[:5, :5])

# Display the first rows of the metadata to examine available annotations
display(meta_df.head())

# Calculate the number of unique compounds present in the filtered dataset
n_compounds = meta_df["cmap_name"].nunique()

# Summarize the number of transcriptional signatures available per compound
compound_counts = meta_df["cmap_name"].value_counts()
display(compound_counts)

---

## Overview of Vitamin D Signature Annotations

### 🧠 General structure and data types

In [ ]:
# Overview of the metadata DataFrame
meta_df.info()

### 📏 Summary statistics for numeric columns

In [ ]:
# Descriptive statistics for numeric variables
print(meta_df.describe())

### 🔣 Cardinality of categorical variables

In [ ]:
# Number of unique values per column
meta_df.nunique().sort_values()

### ⚠️ Missing value check

In [ ]:
# Count of missing values per column
meta_df.isna().sum().sort_values(ascending=False)

### 📌 Signature distribution across key experimental factors

In [ ]:
# Distribution of signatures across cell lines
meta_df["cell_mfc_name"].value_counts()

# Distribution of signatures across exposure durations
meta_df["pert_itime"].value_counts()

This section summarizes the characteristics of the filtered metadata, which includes only signatures related to vitamin D analogs. The dataset was preselected to include:

- **7 compounds** of interest
- **5 human cell lines**
- A single exposure time of **24 hours**

As expected, all signatures reflect this uniformity in treatment duration and compound class. The resulting dataset contains **258 transcriptional signatures**, each associated with a set of experimental and quality control annotations (37 columns in total).

#### 🧪 Quality control and metadata insights

- All signatures have valid values for key fields such as `dose`, `cell line`, and `perturbation type`.
- The `Transcriptional Activity Score (tas)` ranges from 0.01 to 0.64, indicating heterogeneity in transcriptional response even among filtered compounds.
- `ss_ngene` (number of genes significantly changed) varies widely, from 43 to 646.
- The `batch_effect_tstat` and reproducibility scores (`median_recall_*`) show moderate variability, suggesting batch or replicate-level effects worth considering in downstream analysis.

#### ⚠️ Missing values

Most columns are complete. Only `build_name` is entirely missing, and some metrics related to recall and connectivity have partial missingness, which will be handled accordingly if required.

> This metadata summary confirms that the pre-filtering step was correctly applied and that the dataset retains sufficient variability in response quality and intensity to support further exploration.

---

## 🧬 Exploratory Analysis of the Gene Expression Matrix

### 1. Shape and preview

In [ ]:
print("Expression matrix shape (genes × signatures):", exp_df.shape)
print(exp_df.head())

### 2. Summary statistics across all values

In [ ]:
print("Summary statistics for expression values:")
display(exp_df.describe())

### 3. Distribution of all expression values (flattened)


In [ ]:
sns.histplot(exp_df.values.flatten(), bins=100, kde=True)
plt.title("Distribution of Expression Values (All Genes and Signatures)")
plt.xlabel("z-score")
plt.ylabel("Frequency")
plt.show()

### 4. Variance across genes (rows) and signatures (columns)

In [ ]:
gene_var = exp_df.var(axis=1)
sig_var = exp_df.var(axis=0)

print("Gene-wise variance summary:")
display(gene_var.describe())

print("Signature-wise variance summary:")
display(sig_var.describe())



### 5. Check for genes with near-zero variance


In [ ]:
low_var_genes = (gene_var < 1e-5).sum()
print("Number of genes with near-zero variance:", low_var_genes)

This section explores the structure and distribution of the gene expression matrix (`exp_df`) corresponding to the filtered vitamin D-related transcriptional signatures.

### 📐 Matrix dimensions

The expression matrix contains:
- **12,328 genes** (rows)
- **258 signatures** (columns), each matching an experimental condition from the metadata.

Each value represents a **z-score normalized gene expression level**, as provided by the LINCS L1000 pipeline.

### 📊 Summary statistics

A preview of the data shows expected values centered around zero, with both up- and down-regulated genes across conditions. Descriptive statistics confirm this distribution:

- Expression values range from approximately **-8.48 to +9.56**.
- Signature-wise and gene-wise variances are both moderate on average (mean ~0.42).
- **No genes** were found with near-zero variance, indicating that all genes carry some signal and no immediate filtering is needed.

### 📈 Distribution of expression values

The histogram below shows a bell-shaped distribution, consistent with the z-score normalization applied to the data. The central peak is located at 0, and the tails extend in both directions, confirming the presence of genes with both induced and repressed expression across conditions.

>This analysis confirms that the expression matrix is well-structured, contains biologically meaningful variation, and is ready for dimensionality reduction or clustering in subsequent steps.

---


## PCA of Gene Expression Signatures

Principal Component Analysis (PCA) was applied to the gene expression matrix to visualize the structure of the 258 vitamin D-related transcriptional signatures.

This dimensionality reduction technique allows us to:
- Identify potential clustering of signatures by compound or cell line,
- Detect outliers or batch effects,
- Understand how much variance is captured in low-dimensional projections.

The PCA was performed on the **z-score normalized expression matrix** (`exp_df`), already aligned with the metadata.

In [ ]:
# Standardize expression matrix (genes × signatures)
scaler = StandardScaler()
exp_scaled = scaler.fit_transform(exp_df.T)  # Transpose to have samples as rows

# Perform PCA
pca = PCA(n_components=2)
pca_result = pca.fit_transform(exp_scaled)

# Add PCA results to metadata
meta_df["PCA1"] = pca_result[:, 0]
meta_df["PCA2"] = pca_result[:, 1]

# Plot PCA colored by compound
plt.figure(figsize=(8, 6))
sns.scatterplot(
    data=meta_df,
    x="PCA1", y="PCA2",
    hue="cmap_name",
    palette="Set2",
    s=70,
    edgecolor="black",
    alpha=0.8
)
plt.title("PCA of Vitamin D-Related Gene Expression Signatures")
plt.xlabel(f"PCA1 ({pca.explained_variance_ratio_[0]*100:.1f}% variance)")
plt.ylabel(f"PCA2 ({pca.explained_variance_ratio_[1]*100:.1f}% variance)")
plt.legend(title="Compound", bbox_to_anchor=(1.05, 1), loc="upper left")
plt.tight_layout()
plt.show()


#### Interpretation of PCA Results

The PCA projection of the 258 vitamin D-related transcriptional signatures reveals the following:

- **No strong global separation** between compounds is observed, indicating that the overall gene expression profiles are partially overlapping across vitamin D analogs.
- Some mild clustering tendencies appear for specific compounds, such as *maxacalcitol* and *paricalcitol*, which show a few more dispersed or distinctive points—suggesting **unique transcriptomic effects** under certain conditions.
- The explained variance is relatively low (PCA1: 13.6%, PCA2: 5.4%), consistent with high-dimensional biological data. This suggests that a **large number of components** may be needed to capture the full complexity of variation.

> These results suggest that while the compounds share global expression patterns—likely due to their common vitamin D activity—specific outliers may reflect differences in potency, cell type interaction, or downstream pathways activated.

### PCA Colored by Cell Line

To investigate whether cell type explains more variance than compound identity, we re-colored the PCA projection using the `cell_mfc_name` variable.

In [ ]:
# Plot PCA colored by cell line
plt.figure(figsize=(8, 6))
sns.scatterplot(
    data=meta_df,
    x="PCA1", y="PCA2",
    hue="cell_mfc_name",
    palette="Dark2",
    s=70,
    edgecolor="black",
    alpha=0.8
)
plt.title("PCA of Gene Expression Signatures Colored by Cell Line")
plt.xlabel(f"PCA1 ({pca.explained_variance_ratio_[0]*100:.1f}% variance)")
plt.ylabel(f"PCA2 ({pca.explained_variance_ratio_[1]*100:.1f}% variance)")
plt.legend(title="Cell Line", bbox_to_anchor=(1.05, 1), loc="upper left")
plt.tight_layout()
plt.show()


#### Interpretation: PCA Colored by Cell Line

When the same PCA projection is colored by cell line, a clearer structure emerges:

- **PC3 signatures** (orange) form a distinct cluster, mainly in the upper half of the plot, showing a consistent transcriptional pattern across compounds.
- **MCF7, A549, U2OS, and HA1E** signatures largely overlap, suggesting more similar expression responses or lower variability among them.
- This indicates that **cell type has a stronger effect** on the transcriptional landscape than compound identity alone, at least in the space captured by the first two principal components.

> These findings highlight the importance of cellular context in shaping the response to vitamin D analogs, and suggest that stratified analysis by cell line may be necessary in downstream modeling.


## Clustering of Expression Signatures – Heatmap with Dendrogram

To explore the relationships between transcriptional signatures, we used hierarchical clustering on the top 100 most variable genes across all conditions. This clustermap visualizes both sample-sample and gene-gene similarity.

Signatures are annotated by cell line, allowing us to assess whether they cluster according to biological condition.


In [ ]:
# Select top 100 most variable genes
top_genes = exp_df.var(axis=1).sort_values(ascending=False).head(100).index
exp_top = exp_df.loc[top_genes]

# Create color annotations for the columns (signatures)
cell_palette = sns.color_palette("Dark2", meta_df["cell_mfc_name"].nunique())
cell_lut = dict(zip(meta_df["cell_mfc_name"].unique(), cell_palette))
col_colors = meta_df["cell_mfc_name"].map(cell_lut)

# Create clustermap
sns.clustermap(
    exp_top,
    col_cluster=True,
    row_cluster=True,
    col_colors=col_colors,
    cmap="vlag",
    figsize=(12, 10),
    xticklabels=False,
    yticklabels=False,
    dendrogram_ratio=(.1, .2),
    cbar_pos=(0, 0.8, 0.03, 0.18)
)
plt.suptitle("Clustering of Top 100 Variable Genes Across Signatures", y=1.02)
plt.show()


### Interpretation – Clustermap of Top 100 Variable Genes

The hierarchical clustering of gene expression signatures reveals structured patterns of co-expression:

- Signatures from **PC3 cells** cluster tightly, reinforcing their distinct transcriptional response to vitamin D analogs.
- Other cell lines (MCF7, A549, HA1E, U2OS) show more **distributed clustering**, suggesting overlapping but not identical responses.
- Specific gene clusters display coordinated upregulation or downregulation across subsets of conditions, potentially reflecting shared pathways or regulatory modules.

> These results validate and extend the PCA observations, showing that cell line identity remains a strong driver of expression variability, even at the level of the most variable genes.


## UMAP of Gene Expression Signatures

To complement the PCA and clustering analysis, we applied **Uniform Manifold Approximation and Projection (UMAP)** to the expression matrix. UMAP is a non-linear dimensionality reduction technique that preserves both local and global structure in the data.

This method is particularly useful for detecting subtle patterns and **natural groupings** that may not be captured by PCA. Here, we visualize the transcriptional signatures in a 2D space using UMAP, colored by **cell line identity**.


In [ ]:
# Import UMAP
import umap.umap_ as umap

# Scale the data (use same scaling as in PCA)
scaler = StandardScaler()
exp_scaled = scaler.fit_transform(exp_df.T)

# Run UMAP
umap_model = umap.UMAP(n_neighbors=15, min_dist=0.1, random_state=42)
umap_result = umap_model.fit_transform(exp_scaled)

# Save results in metadata
meta_df["UMAP1"] = umap_result[:, 0]
meta_df["UMAP2"] = umap_result[:, 1]

In [ ]:
# Plot UMAP por cell line
plt.figure(figsize=(8, 6))
sns.scatterplot(
    data=meta_df,
    x="UMAP1", y="UMAP2",
    hue="cell_mfc_name",
    palette="Dark2",
    s=70,
    edgecolor="black",
    alpha=0.8
)
plt.title("UMAP of Gene Expression Signatures by Cell Line")
plt.legend(title="Cell Line", bbox_to_anchor=(1.05, 1), loc="upper left")
plt.tight_layout()
plt.show()


#### Interpretation of UMAP Results

The UMAP projection of gene expression signatures across selected cell lines reveals the following:

- **Clear separation** is observed for the PC3 cell line, which forms a distinct and compact cluster—suggesting a **consistent and unique transcriptomic response** compared to the other lines.
- **Partial overlap** is present among MCF7, A549, HA1E, and U2OS, indicating **shared or more variable expression profiles**, possibly due to common pathways or cell-type similarities.
- UMAP maintains meaningful **local structure** in the data, capturing both discrete and continuous relationships between samples.
- Unlike PCA, UMAP is non-linear and does not provide explained variance metrics, but its **visual separability** implies underlying biological differences.

> These observations highlight how transcriptional responses to perturbagens can vary by cell type. While some cell lines (e.g., PC3) respond in a well-defined manner, others exhibit broader variability, potentially reflecting differences in lineage, basal gene expression, or compound sensitivity.

---

## Distribution of Experimental Metrics by Compound and Cell Line

To evaluate the consistency and variability of transcriptional responses, we visualize three key metrics from the metadata:

- `tas`: Transcriptional Activity Score
- `ss_ngene`: Number of significantly changed genes
- `cc_q75`: 75th percentile of replicate correlation (quality control)

These metrics reflect signal strength, gene-level impact, and reproducibility, respectively. We compare their distributions across compounds (`cmap_name`) and cell lines (`cell_mfc_name`) to assess whether specific treatments or cellular contexts show stronger or more variable responses.


### 🔹 Distribution by Compound

In [ ]:
# Define metrics to plot
metrics = ["tas", "ss_ngene", "cc_q75"]
titles = ["Transcriptional Activity Score (TAS)",
          "Number of Significantly Changed Genes (ss_ngene)",
          "75th Percentile of Replicate Correlation (cc_q75)"]

# Create a single figure with 3 subplots (stacked vertically)
fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(7, 12), sharex=True)

for i, metric in enumerate(metrics):
    sns.boxplot(
        data=meta_df,
        x="cmap_name",
        y=metric,
        hue="cmap_name", 
        palette="Set2",
        legend=False,
        ax=axes[i]
    )
    
    axes[i].set_title(titles[i])
    axes[i].set_ylabel(metric.upper())
    axes[i].set_xlabel("")
    axes[i].tick_params(axis='x', rotation=45)

# Set common X label on the last plot
axes[-1].set_xlabel("Compound (cmap_name)")

plt.tight_layout()
plt.show()


#### 🧠 Interpretation – Metrics by Compound

The boxplots of `tas`, `ss_ngene`, and `cc_q75` across vitamin D-related compounds reveal moderate variability, with no single compound showing consistently superior performance across all metrics.

- **TAS** values are broadly comparable across compounds, suggesting similar overall transcriptional activity. Slightly higher medians are observed for *tacalcitol*, *seocalcitol*, and *paricalcitol*.
- **ss_ngene**, representing the number of significantly changed genes, shows the highest dispersion for *paricalcitol*, indicating strong but variable gene-level responses.
- **cc_q75**, a proxy for replicate consistency, is relatively balanced, though *maxacalcitol* and *paricalcitol* exhibit marginally higher median values.

> These results suggest that all compounds induce measurable transcriptional activity, but the **strength and reproducibility** of responses vary slightly across analogs.


### 🔹 Distribution by Cell Line

In [ ]:
# Define metrics to plot
metrics = ["tas", "ss_ngene", "cc_q75"]
titles = ["Transcriptional Activity Score (TAS)",
          "Number of Significantly Changed Genes (ss_ngene)",
          "75th Percentile of Replicate Correlation (cc_q75)"]

# Create a single figure with 3 subplots (stacked vertically)
fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(6, 12), sharex=True)

for i, metric in enumerate(metrics):
    sns.boxplot(
        data=meta_df,
        x="cell_mfc_name",
        y=metric,
        hue="cell_mfc_name", 
        palette="Dark2",
        legend=False,
        ax=axes[i]
    )
    
    axes[i].set_title(titles[i])
    axes[i].set_ylabel(metric.upper())
    axes[i].set_xlabel("")
    axes[i].tick_params(axis='x', rotation=45)

# Set common X label on the last plot
axes[-1].set_xlabel("Cell Line (cell_mfc_name)")

plt.tight_layout()
plt.show()

### 🧬 Cell Line Comparison of Transcriptomic Response Metrics

The distribution of transcriptional activity metrics across cell lines reveals that **cellular context strongly shapes the magnitude and reproducibility of responses to vitamin D analogs**.

- **PC3 cells** stand out with **higher median TAS**, **more significantly altered genes (ss_ngene)**, and **stronger replicate correlation (cc_q75)**, indicating a robust and consistent transcriptional signature.
- Nonetheless, **other cell lines such as MCF7, U2OS, A549, and HA1E also show meaningful transcriptomic changes**, with **median `ss_ngene` values often exceeding 100–150 genes**, a level considered biologically significant in many studies.
- **Higher variability** in `cc_q75` across some lines (e.g., A549, HA1E) might reflect lower reproducibility or more heterogeneous responses—important to keep in mind for downstream modeling or interpretation.

> These results support the notion that **vitamin D analogs produce broad transcriptional effects across multiple cancer-related cell types**, with **PC3 being particularly responsive**, but by no means the only relevant model.


### 📊 Distribution of Transcriptomic Signatures per Cell Line

Before diving deeper into transcriptional patterns, it's essential to assess the **representation of each cell line** in our dataset. A highly responsive cell line may appear to perform better simply due to a **larger number of replicates** or signatures. This plot helps evaluate whether the observed trends reflect biological differences or data imbalance.

#### 📌 Count the number of transcriptomic signatures available per cell line


In [ ]:
# Count how many signatures exist for each cell line
cell_line_counts = meta_df['cell_mfc_name'].value_counts().reset_index()
cell_line_counts.columns = ['Cell Line', 'Number of Signatures']

# Plot the distribution using a barplot (compatible with future Seaborn versions)
plt.figure(figsize=(6, 4))
sns.barplot(
    data=cell_line_counts,
    x='Cell Line',
    y='Number of Signatures',
    hue='Cell Line',              
    palette='Set2',
    legend=False                  
)

# Add labels and formatting
plt.title("Number of Transcriptomic Signatures per Cell Line")
plt.ylabel("Number of Signatures")
plt.xlabel("Cell Line")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

The distribution of vitamin D-related transcriptomic signatures is relatively balanced across the main cell lines, with **MCF7, PC3, A549, and HA1E** each contributing over 50 signatures. **U2OS** has fewer profiles (~20), which may slightly limit statistical power for this cell type.

This coverage is consistent with the compound × cell line matrix, where *calcitriol* is the most widely tested analog across all lines. Other compounds like *tacalcitol* or *seocalcitol* show more limited sampling, but still span multiple cell types.

> Therefore, the observed transcriptional differences between cell lines are **not simply due to data imbalance**, and likely reflect genuine biological context effects.

### 🔬 Most Strongly Modulated Genes Across All Signatures

To identify the genes most responsive to vitamin D analogs, we computed the **mean absolute z-score** for each gene across all 258 transcriptional signatures.

This metric captures the overall magnitude of transcriptional change, regardless of direction (up- or down-regulation). Genes with higher mean |z| values are considered **globally more responsive** to vitamin D perturbation.

#### 📌 Identify the top 20 most strongly modulated genes across all signatures

In [ ]:
# Load gene information table
geneinfo_df = pd.read_csv("../raw_data/geneinfo_beta.txt", sep="\t", low_memory=False)

# Compute the mean absolute z-score for each gene
mean_abs_z = exp_df.abs().mean(axis=1).sort_values(ascending=False)

# Select the top 20 most modulated genes
top_genes = mean_abs_z.head(20)

# Map gene IDs (rid) to gene symbols
top_genes_named = (
    top_genes.reset_index()
    .merge(geneinfo_df[["gene_id", "gene_symbol"]], left_on="rid", right_on="gene_id", how="left")
    .set_index("gene_symbol")
    .sort_values(by=0, ascending=False)
)

# Display the resulting table
print(top_genes_named)

# Plot with gene symbols as Y-axis labels
plt.figure(figsize=(8, 6))
sns.barplot(
    x=top_genes_named[0],
    y=top_genes_named.index,
    hue=top_genes_named.index,  # assign y-axis variable to hue
    palette="viridis",
    dodge=False,
    legend=False
)
plt.xlabel("Mean |z-score| across all signatures")
plt.ylabel("Gene Symbol")
plt.title("Top 20 Most Modulated Genes by Vitamin D Analog Exposure")
plt.tight_layout()
plt.show()

#### 🧠 Interpretation – Top Modulated Genes

The analysis of mean absolute z-scores across all 258 vitamin D-related transcriptional signatures highlights several genes with consistently strong modulation:

- **IGFBP3** and **DDIT4** top the list, both of which are known to be regulated by vitamin D and associated with apoptosis, cellular stress, and proliferation control.
- Other highly responsive genes include **TXNRD1** (oxidative stress), **NFKBIA** (inflammation), **PHGDH** (serine biosynthesis), and **SPP1** (osteopontin), all of which play roles in cellular homeostasis and cancer biology.
- The presence of **C2CD2**, **TSKU**, and other less characterized genes suggests possible **novel regulatory effects** of vitamin D analogs worthy of further investigation.

> These findings support the hypothesis that vitamin D analogs elicit biologically relevant transcriptional responses, impacting both well-known and potentially novel gene targets.


### 🔥 Heatmap of Top 20 Most Modulated Genes

To explore how the most responsive genes behave across all transcriptional signatures, we created a heatmap of z-score expression values for the **top 20 most modulated genes**. This visualization reveals potential clustering patterns among compounds, cell lines, and gene regulation modes.


In [ ]:
# Extract expression data for the top 20 genes (using gene_id)
top_gene_ids = top_genes_named["gene_id"]
heatmap_df = exp_df.loc[top_gene_ids]

# Optional: replace gene_id by gene_symbol for readability
heatmap_df.index = top_genes_named.index  # these are gene symbols

# Create column color annotations for cell line
cell_palette = sns.color_palette("Dark2", meta_df["cell_mfc_name"].nunique())
cell_lut = dict(zip(meta_df["cell_mfc_name"].unique(), cell_palette))
col_colors = meta_df["cell_mfc_name"].map(cell_lut)

# Plot clustermap
sns.clustermap(
    heatmap_df,
    col_cluster=True,
    row_cluster=True,
    col_colors=col_colors,
    cmap="vlag",
    figsize=(14, 10),
    xticklabels=False,
    yticklabels=True,
    dendrogram_ratio=(.1, .2),
    cbar_pos=(0, 0.8, 0.03, 0.18)
)
plt.suptitle("Expression Heatmap – Top 20 Most Modulated Genes", y=1.02)
plt.show()


### 🧬 Interpretation – Heatmap of Top 20 Most Modulated Genes

The heatmap reveals **coherent transcriptional response patterns** among the top 20 most modulated genes across vitamin D-related signatures.

- Distinct clusters of gene activation (red) and repression (blue) are visible, indicating **co-regulation** and potential **shared pathways**.
- Several genes, such as **IGFBP3**, **DDIT4**, and **TXNRD1**, show strong and consistent upregulation in subsets of signatures.
- Column annotations by cell line confirm that some cell types (e.g., **PC3**) exhibit **stronger and more consistent responses**, aligning with previous PCA and metric-based observations.

> These expression profiles suggest that both compound and cellular context shape the transcriptional landscape, and that a subset of genes may serve as robust indicators of vitamin D activity.


### 🧪 Activity Intensity by Compound and Cell Line

To summarize the strength of transcriptional responses across experimental conditions, we computed the average **Transcriptional Activity Score (TAS)** for each combination of **compound** and **cell line**.

This heatmap highlights which cell-compound contexts exhibit the strongest overall modulation, guiding future analyses or experimental prioritization.


In [ ]:
# Create pivot table with mean TAS per compound × cell line
tas_pivot = meta_df.pivot_table(
    index="cell_mfc_name",
    columns="cmap_name",
    values="tas",
    aggfunc="mean"
)

# Plot heatmap
plt.figure(figsize=(10, 6))
sns.heatmap(tas_pivot, cmap="YlGnBu", annot=True, fmt=".2f", linewidths=0.5, cbar_kws={"label": "Mean TAS"})
plt.title("Mean Transcriptional Activity Score (TAS) by Compound and Cell Line")
plt.xlabel("Compound")
plt.ylabel("Cell Line")
plt.tight_layout()
plt.show()


#### 🧬 Observations on Transcriptional Activity Score (TAS)

- The **PC3** cell line shows the **highest transcriptional response** across all Vitamin D analogs, with TAS values consistently above 0.30. This suggests that prostate cancer cells may be particularly sensitive to modulation by these compounds.

- **Ergocalcitriol, paricalcitol, and seocalcitol** appear to induce **stronger transcriptional activity overall**, especially in PC3, pointing to potential differences in potency or mechanism compared to calcitriol.

- In contrast, **U2OS (osteosarcoma)** and **A549 (lung cancer)** lines show **weaker transcriptional responses**, with TAS values typically below 0.20, indicating limited gene expression modulation under the tested conditions.

- **MCF7** (breast cancer) and **HA1E** (kidney epithelial) display **moderate and variable TAS values**, suggesting context-dependent sensitivity that may reflect differences in Vitamin D receptor expression or downstream signaling.

> These patterns highlight the importance of cell-type specificity when evaluating the transcriptomic impact of Vitamin D analogs and may inform future therapeutic targeting strategies.

### 🧠 Comparison of Correlation Analyses: Treatment Similarity vs. Gene Co-regulation

To gain a more complete understanding of the transcriptional effects of vitamin D analogs, we performed **two complementary correlation analyses** on the expression matrix.


#### 🔵 1. Gene Co-regulation Across Vitamin D Treatments

**Analysis:**  
We computed the **Pearson correlation between genes** (rows of the expression matrix), across all vitamin D signatures.

**Question addressed:**  
*Which genes tend to behave similarly in response to vitamin D analogs?*

**Biological interpretation:**
- Co-regulated genes may share **regulatory elements**, belong to the same **pathways**, or be **core effectors** of the vitamin D response.
- Highly correlated genes might be **redundant**, while inversely correlated ones may represent **divergent pathways**.
- This helps identify **candidate biomarkers** or **functional modules** activated by treatment.

In [ ]:
# Cargar tabla de información de genes (si no está cargada aún)
geneinfo_df = pd.read_csv("../raw_data/geneinfo_beta.txt", sep="\t", low_memory=False)

# Calcular z-score absoluto promedio por gen
mean_abs_z = exp_df.abs().mean(axis=1).sort_values(ascending=False)

# Seleccionar los 20 genes más modulados
top_genes = mean_abs_z.head(20)

# Obtener nombres de genes (gene_symbol)
top_genes_named = (
    top_genes.reset_index()
    .merge(geneinfo_df[["gene_id", "gene_symbol"]], left_on="rid", right_on="gene_id", how="left")
    .set_index("gene_symbol")
    .sort_values(by=0, ascending=False)
)

# Filtrar matriz de expresión por los top genes
exp_top = exp_df.loc[top_genes_named["rid"]]

# Renombrar los índices de fila (genes) con símbolos
exp_top.index = top_genes_named.index

# Alinear metadata con columnas de expresión
meta_aligned = meta_df.loc[exp_top.columns]

# Agrupar por nombre de compuesto y promediar perfiles
exp_top_avg = exp_top.T.groupby(meta_aligned["cmap_name"]).mean().T

# Calcular correlación entre compuestos
compound_corr = exp_top_avg.T.corr(method="pearson")

# Plot del heatmap
plt.figure(figsize=(12, 10))
sns.heatmap(
    compound_corr,
    annot=True,
    fmt=".2f",
    cmap="vlag",
    vmin=0,
    vmax=1,
    square=True,
    linewidths=0.5,
    cbar_kws={"label": "Pearson Correlation"}
)
plt.title("Gene Co-regulation Based on Responses to Vitamin D Analogs (Top 20 Genes)", fontsize=14)
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()


#### *🔬 Key Insights from Gene Co-regulation Heatmap*

This heatmap reveals patterns of **co-regulation among the top 20 genes** most modulated by vitamin D analogs.  
Notably:

- Several gene pairs show **strong positive correlation** (e.g., `MMP1` and `NPC1`), suggesting shared regulatory mechanisms or involvement in similar pathways.
- A few genes, such as `MTHFD2` and `PRSS23`, exhibit **anti-correlated patterns**, potentially reflecting opposing functional roles.
- These insights can help prioritize **candidate genes for downstream pathway analysis or biomarker development**.


### 🔴 2. Treatment Similarity Based on Gene Expression Profiles

**Analysis:**  
We computed the **Pearson correlation between treatment signatures** (columns of the expression matrix), using the top 20 most modulated genes.

**Question addressed:**  
*How similar are the transcriptomic responses induced by different vitamin D analogs?*

**Biological interpretation:**
- Compounds that cluster together in this analysis likely activate **similar transcriptional programs**.
- Differences may reflect **distinct receptor affinity**, **cellular uptake**, or **mechanistic selectivity**.
- This is useful to identify **functionally similar compounds**, even when their chemical structure differs — a principle applied in **drug repurposing**.

---

In [ ]:
# Select top 20 most modulated genes (based on mean absolute z-score)
top_gene_ids = top_genes.index  # gene_id format

# Extract expression data for these genes
exp_top = exp_df.loc[top_gene_ids]

# Retrieve gene symbols for labeling
gene_symbols = top_genes_named.index.tolist()

# Compute Pearson correlation between gene expression profiles
gene_corr = exp_top.T.corr(method="pearson")

# Plot heatmap of gene-gene correlations
plt.figure(figsize=(10, 8))
sns.heatmap(
    gene_corr,
    cmap="coolwarm",
    center=0,
    xticklabels=gene_symbols,
    yticklabels=gene_symbols,
    annot=True,
    fmt=".2f",
    square=True,
    linewidths=0.5,
    cbar_kws={"label": "Pearson Correlation"}
)
plt.title("Correlation Between Top 20 Modulated Genes", fontsize=14)
plt.xticks(rotation=45, ha="right")
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()


#### *🧬 Gene-Gene Correlation Across All Vitamin D Signatures*

This plot shows the **pairwise Pearson correlation** between the top 20 modulated genes across all vitamin D-related signatures (i.e., individual experimental conditions).

- Genes such as `TXNRD1`, `C2CD2`, and `NFKBIA` show **consistent co-modulation patterns**, suggesting robust responses across conditions.
- Strong negative correlations (e.g., between `PRSS23` and `TXNRD1`) may indicate **functionally opposing transcriptional programs**.
- These patterns reflect the **global coordination or antagonism** of gene responses to vitamin D analog exposure, independent of specific compounds.

> This heatmap helps assess how **stable or divergent gene relationships** are across the entire dataset, which can be crucial for identifying network modules or regulatory axes.


### ✅ Why both analyses are important

| Analysis                                | Focus                       | Key Insight                        |
|----------------------------------------|-----------------------------|------------------------------------|
| **Treatment Similarity** (`df.corr()`) | Across treatments           | Groups compounds by response       |
| **Gene Co-regulation** (`df.T.corr()`) | Across genes                | Reveals functional gene modules    |

By running both, we explore **how compounds behave**, and **what genes co-operate**, providing a more holistic view of the transcriptional landscape.

